In [1]:
import numpy as np
import healpy as hp
from lsst.sims.skybrightness import skyModel
import warnings
table = np.load("mjdRaw.npz")

In [2]:
def getRaw(mjd,delta_t=0.007):
    """
    Determine the rawSeeing at a given time from the OpSim Database.
    
    Parameters
    __________
    
    mjd: float
        Modified Julian Date of exposure.
        
    delta_t: float (0.007)
        The maximum difference in time between given mjd and 
        mjd from database from which rawSeeing will be interpolated, days.
    
    
    Returns
    __________
    
    rawSeeing: float
        An estimate of the rawSeeing at mjd. If LSST not observing at mjd, returns NaN.
        
    """
    
    indAfter = np.min(np.where(table['mjd']>mjd)[0])
    deltatAfter = table['mjd'][indAfter]-mjd
    rawAfter = table['rawSee'][indAfter]
    
    indBefore = np.max(np.where(table['mjd']<mjd)[0])
    deltatBefore = mjd-table['mjd'][indBefore]
    rawBefore = table['rawSee'][indBefore]
    
    if deltatAfter<delta_t and deltatBefore<delta_t:
        return rawBefore + (deltatBefore/ (deltatBefore+deltatAfter))*(rawAfter-rawBefore)
    if deltatAfter<delta_t:
        return rawAfter
    if deltatBefore<delta_t:
        return rawBefore
    else:
        warnings.warn('LSST not viewing at indicated expMJD')
        return np.nan

In [3]:
def getAirmass(ra,dec,mjd):
    """
    Determine the airmass at given coordinates and time from ESO model.
    
    Parameters
    __________
    
    ra: array_like
        Right ascension, radians.
        
    dec: array_like
        Declinaton, radians.
    
    mjd: float
        Modified Julian Date.
    
    
    Returns
    __________
    
    airmass: ndarray or scalar
        The airmass in the form in which ra and dec are provided. 
        If airmass > 2.5, returns NaN.
        
    """
    
    altAz = skyModel.stupidFast_RaDec2AltAz(np.atleast_1d(ra),np.atleast_1d(dec),-0.52786436029,-1.23480997381,mjd)
    alt = altAz[0]
    airmass = 1./np.cos(np.pi/2.-alt)
    highXInd = np.where(airmass>2.5)[0]
    lowXInd = np.where(airmass<1)[0]
    airmass[highXInd] = np.nan
    airmass[lowXInd] = np.nan
    
    return airmass

In [4]:
def getSeeing(filt,ra,dec,mjd):
    """
    Determine the seeing.
    
    Parameters
    __________
    
    filt: string
        The LSST filter in use: 'u' 'g' 'r' 'i' 'z' 'y'.
        
    ra: array_like
        Right ascension, radians.
        
    dec: array_like
        Declinaton, radians.
    
    mjd: float
        Modified Julian Date.
    
    
    Returns
    __________
    
    seeing: ndarray or scalar
        The seeing in the form in which ra and dec are provided. 
        At locations where airmass>2.5, returns NaN.
        
    """
    
    filterWave = {'u': 367.0, 'g': 482.5, 'r': 622.2, 'i': 754.5, 'z': 869.1, 'y': 971.0}
    wavelength = filterWave[filt]

    rawSeeing = getRaw(mjd)
    
    airmass = getAirmass(ra,dec,mjd)
    
    telSeeing = 0.25
    opticalDesign = 0.08
    cameraSeeing = 0.30
    fwhm_sys = np.sqrt(telSeeing**2 + opticalDesign**2 + cameraSeeing**2) * np.power(airmass, 0.6)
    fwhm_atm = rawSeeing * np.power(airmass, 0.6) * np.power(500.0 / wavelength, 0.3)
    fwhmEff = 1.16 * np.sqrt(fwhm_sys**2 + 1.04*fwhm_atm**2)
    return fwhmEff

In [5]:
def getSkyBright(filt,ra,dec,mjd):
    """
    Determine the sky brightness.
    
    Parameters
    __________
    
    filt: string
        The LSST filter in use: 'u' 'g' 'r' 'i' 'z' 'y'.
        
    ra: array_like
        Right ascension, radians.
        
    dec: array_like
        Declinaton, radians.
    
    mjd: float
        Modified Julian Date.
    
    
    Returns
    __________
    
    skyBrightness: ndarray or scalar
        The sky brightness in the form in which ra and dec are provided, Mag. 
        At locations where airmass > 2.5, returns NaN.
        
    """
    
    ra = np.atleast_1d(ra)
    dec = np.atleast_1d(dec)
    
    airmass = getAirmass(ra,dec,mjd)
    mags = np.full_like(airmass, np.nan)
    
    goodInd = np.where(np.isnan(airmass)==False)[0]
    if goodInd.size>0:
        sm = skyModel.SkyModel(mags = True)
        sm.setRaDecMjd(ra[goodInd], dec[goodInd], mjd)
        mags[goodInd] = sm.returnMags()[filt][0]

    return mags

In [6]:
def get5sig(filt,ra,dec,mjd,tvis = 30):
    """
    Determine the five-sigma depth for a point source.
    
    Parameters
    __________
    
    filt: string
        The LSST filter in use: 'u' 'g' 'r' 'i' 'z' 'y'.
        
    ra: array_like
        Right ascension, radians.
        
    dec: array_like
        Declinaton, radians.
    
    mjd: float
        Modified Julian Date.
        
    tvis: float (30)
        The exposure time for the observation, seconds.
    
    
    Returns
    __________
    
    m5: ndarray or scalar
        The five-sigma depth in the form in which ra and dec are provided, Mag. 
        At locations where airmass > 2.5, returns NaN.
        
    """
    
    seeing = getSeeing(filt,ra,dec,mjd)
    skyBright = getSkyBright(filt,ra,dec,mjd)
    airmass = getAirmass(ra,dec,mjd)
    
    cmdict = {'u': 23.09, 'g': 24.42, 'r': 24.44, 'i': 24.32, 'z': 24.16, 'y': 23.73}
    cm = cmdict[filt]
    
    kmdict = {'u': 0.491, 'g': 0.213, 'r': 0.126, 'i': 0.096, 'z': 0.067, 'y': 0.17}
    km = kmdict[filt]
    
    m5 = cm + 0.5*(skyBright-21) + 2.5*np.log10(0.7/seeing)+1.25*np.log10(tvis/30)-km*(airmass-1)
    
    return m5